In [23]:
def csv_to_pandas(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df

def get_needed_coloumns(objects,cols_needed):
    df_all = pd.DataFrame()
    df_all = pd.concat([csv_to_pandas(obj.key) for obj in objects],ignore_index=True)
    df_all = df_all.loc[:,cols_needed]
    df_all.dropna(inplace=True)
    return df_all

def get_needed_coloumns_xetra_changes(df_all):
    df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')
    df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')
    df_all = df_all.groupby(['ISIN','Date'], as_index=True)
    df_all = df_all.agg(
        opening_price_euro=('opening_price','min'), 
        closing_price_euro=('closing_price','min'), 
        minimum_price_euro=('MinPrice','min'), 
        maxmimum_price_euro=('MaxPrice','max'), 
        daily_traded_volume=('TradedVolume','sum') )
    df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_euro'].shift(1)
    df_all['change_prev_closing_precent'] = (df_all['closing_price_euro'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
    df_all.drop(columns=['closing_price_euro'], inplace=True)
    df_all = df_all.round(decimals=2)
    return df_all

def upload_s3(bucket_target, output_buffer):
    bucket_target.put_object(Body=output_buffer.getvalue(),Key=keys)

In [22]:
import boto3
import pandas as pd
from io import StringIO,BytesIO # to read csv using s3 storage
from datetime import datetime, timedelta 
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

arg_date = '2022-12-28'
suorce_date_format = '%Y-%m-%d'
duet_xetra_source = 'xetra-1234'
target_bucket = 'xetra-destination'
keys = 'parquet/xetra_daily_report_'+datetime.today().strftime('%y%m%d_%H%M%S')+'.parquet'
cols_needed = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume', 'NumberOfTrades']
az_des_con_str = 'DefaultEndpointsProtocol=https;AccountName=xetradestination;AccountKey=jHuTzDSICJbvjv/YKkQescXmPJg8Gntu/U1/P36fXqssoTAjPj7LCyo/lj1tTcjeSBaoY2IzFyKe+AStvRyTtQ==;EndpointSuffix=core.windows.net'

def main():
    arg_date_dt = datetime.strptime(arg_date,suorce_date_format ).date() - timedelta(days=1) 
    output_buffer = BytesIO()
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(duet_xetra_source) 
    bucket_target = s3.Bucket(target_bucket)
    objects = [obj for obj in bucket.objects.all() if datetime.strptime(str(obj.key).split('/')[0], '%Y-%m-%d').date() >= arg_date_dt]

    df_all = get_needed_coloumns(objects,cols_needed)
    df_all = get_needed_coloumns_xetra_changes(df_all)
    df_all.to_parquet(output_buffer, index=False)
    upload_s3(bucket_target, output_buffer)


KeyboardInterrupt



In [8]:
import boto3
s3 = boto3.resource('s3')
import pandas as pd
from io import StringIO,BytesIO
df_report = []
target_bucket = 'xetra-destination'
bucket_target = s3.Bucket(target_bucket)
for obj in bucket_target.objects.all():
    print(obj.key)
    parquet_obj = bucket_target.Object(key=obj.key).get().get('Body').read()
    data = BytesIO(parquet_obj)
    df_report.append({obj.key:pd.read_parquet(data)})


parquet/xetra_daily_report_221230_134318.parquet
parquet/xetra_daily_report_230101_151136.parquet
parquet/xetra_daily_report_230101_161249.parquet
parquet/xetra_daily_report_230101_162851.parquet
parquet/xetra_daily_report_230101_174938.parquet
parquet/xetra_daily_report_230101_190458.parquet
parquet/xetra_daily_report_230101_193836.parquet
[{'parquet/xetra_daily_report_221230_134318.parquet':        opening_price_euro  minimum_price_euro  maxmimum_price_euro  \
0                   36.10               36.10                37.70   
1                   36.60               35.75                36.70   
2                   36.60               35.75                36.70   
3                   36.60               35.75                36.70   
4                    7.90                7.86                 8.36   
...                   ...                 ...                  ...   
12838               34.29               34.06                36.50   
12839                3.44                3.